In [1]:
from modules.base import *

#### 메모리 사용하기
- 메모리 종류는 2가지

- short term memory : 동일 thread 내에서 메모리 공유    
    - 사용 라이브러리 : `from langgraph.checkpoint.memory import MemorySaver`

    - build compile 시 checkpointer 인자로 사용
    
- long term memory : 동일 사용자 내에서 메모리 공유
    - 사용 라이브러리 : `from langgraph.store.memory import InMemoryStore`

    - 구조는 [namespace(directory, userid) -> key -> value]

    - build compile 시 store 인자로 사용


![Sample Image](../public/short_term_long_term.png)

---

##### 1. ShortTerm 테스트

In [2]:
@trace_function(only_func_name=False, print_json_format=True)
def node_answer(state:MessagesState)->MessagesState:
    return {"messages": [llm.invoke(state["messages"])]}
builder = StateGraph(MessagesState)
builder.add_node("node_answer", node_answer)
builder.add_edge(START, "node_answer")
builder.add_edge("node_answer", END)
ShortTermMemory = MemorySaver()
graph = builder.compile(checkpointer=ShortTermMemory)

config = {"configurable": {"thread_id": "initial_chat", 
                           "user_id": "changwoo"}}

In [3]:
graph.invoke({"messages":"안녕 나는 창우라고해"}, config)


🚀 Passing Through [node_answer] ..

#### [Input State]
args: [
    {
        "messages": [
            {
                "content": "안녕 나는 창우라고해",
                "additional_kwargs": {},
                "type": "human",
                "name": null,
                "id": "ef8f3c61-a26c-472f-af6d-b7c032ae6dae",
                "example": false
            }
        ]
    }
]

#### [Output State]
result: {
    "messages": [
        {
            "content": "안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?",
            "additional_kwargs": {
                "refusal": null
            },
            "type": "ai",
            "name": null,
            "id": "run-a6c4c00a-e659-480a-8acd-7d808212d769-0",
            "example": false,
            "tool_calls": [],
            "invalid_tool_calls": []
        }
    ]
}


{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='ef8f3c61-a26c-472f-af6d-b7c032ae6dae'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 14, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7d65f1a5b', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6c4c00a-e659-480a-8acd-7d808212d769-0', usage_metadata={'input_tokens': 14, 'output_tokens': 22, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [4]:
graph.invoke({"messages":"내 이름이 뭐라고?"}, config)


🚀 Passing Through [node_answer] ..

#### [Input State]
args: [
    {
        "messages": [
            {
                "content": "안녕 나는 창우라고해",
                "additional_kwargs": {},
                "type": "human",
                "name": null,
                "id": "ef8f3c61-a26c-472f-af6d-b7c032ae6dae",
                "example": false
            },
            {
                "content": "안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?",
                "additional_kwargs": {
                    "refusal": null
                },
                "type": "ai",
                "name": null,
                "id": "run-a6c4c00a-e659-480a-8acd-7d808212d769-0",
                "example": false,
                "tool_calls": [],
                "invalid_tool_calls": []
            },
            {
                "content": "내 이름이 뭐라고?",
                "additional_kwargs": {},
                "type": "human",
                "name": null,
                "id": "9fbe4a5f-3a3c-4e06-848d-0d37

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='ef8f3c61-a26c-472f-af6d-b7c032ae6dae'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 14, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7d65f1a5b', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6c4c00a-e659-480a-8acd-7d808212d769-0', usage_metadata={'input_tokens': 14, 'output_tokens': 22, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='내 이름이 뭐라고?', additional_kwargs={}, response_metadata={}, id='9fbe4a5f-3a3c-4e06

주의할 점은 `단순히 graph만 새로 build하면 여전히 정보를 기억`한다는 점

- 아래 3가지 방식중에 하나로 초기화 해주어야 함
    - option 1. `ShortTermMemory = MemorySaver()` 재 선언후 graph build
    - option 2. 세션 새로 시작
    - option 3. config의 thread_id 값 변경

- LongTermMemory 도 동일함

In [6]:
# 초기화안하고 graph만 다시 build 할경우 기록남아있음.
# ShortTermMemory = MemorySaver() 
graph = builder.compile(checkpointer=ShortTermMemory)
graph.invoke({"messages":"내 이름이 뭐라고?"}, config)


🚀 Passing Through [node_answer] ..

#### [Input State]
args: [
    {
        "messages": [
            {
                "content": "안녕 나는 창우라고해",
                "additional_kwargs": {},
                "type": "human",
                "name": null,
                "id": "8b1e414f-227e-4c85-9eca-b8b5174ea026",
                "example": false
            },
            {
                "content": "안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?",
                "additional_kwargs": {
                    "refusal": null
                },
                "type": "ai",
                "name": null,
                "id": "run-5fd79542-c987-46a7-a746-d361c68dbaec-0",
                "example": false,
                "tool_calls": [],
                "invalid_tool_calls": []
            },
            {
                "content": "내 이름이 뭐라고?",
                "additional_kwargs": {},
                "type": "human",
                "name": null,
                "id": "b7012b75-0242-4de0-bc3b-418cd80

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='8b1e414f-227e-4c85-9eca-b8b5174ea026'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-5fd79542-c987-46a7-a746-d361c68dbaec-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='내 이름이 뭐라고?', additional_kwargs={}, response_metadata={}, id='b7012b75-0242-4de0-bc

##### 2. LongTerm 테스트

단순 테스트

In [7]:
directory = "memories"
userid = "changwoo"
key = "chat_user_memory"
value = {"나이" : "29", 
         "전공": "Computer Engineering",
         "취미": "working out",
         "좋아하는 음식": "pork"}

LongTermMemory = InMemoryStore()

# 값 넣기
LongTermMemory.put(namespace=(directory, userid),
                   key=key,  
                   value=value)

# 값 가져오기 (1) - search 함수 사용 (리턴타입 : list)
memory = LongTermMemory.search((directory, userid))
print(memory[0].dict()['value'])


# 값 가져오기 (2) - get 함수 사용 (리턴타입 : dict)
memory = LongTermMemory.get((directory, userid),key)
print(memory.dict()['value'])

{'나이': '29', '전공': 'Computer Engineering', '취미': 'working out', '좋아하는 음식': 'pork'}
{'나이': '29', '전공': 'Computer Engineering', '취미': 'working out', '좋아하는 음식': 'pork'}


그래프를 활용한 테스트

In [8]:
prompt_config = ConfigDict()
prompt_config.answer_prompt = """당신은 사용자의 정보를 기억하고, 이를 활용해 개인화된 정보를 제공하는 유용한 조력자입니다. 
사용자에 대한 기억이 있다면 이를 사용해 응답을 맞춤화하세요.
다음은 사용자의 기억입니다 (비어 있을 수도 있습니다): {memory}"""

prompt_config.create_memory_prompt ="""당신은 사용자의 응답을 개인화하기 위해 사용자에 대한 정보를 수집하고 있습니다.

현재 사용자 정보:
{memory}

지침:
1. 아래의 채팅 기록을 주의 깊게 검토하세요.
2. 사용자에 대한 새로운 정보를 식별하세요. 예를 들면:
   - 개인 정보 (이름, 위치 등)
   - 선호 사항 (좋아하는 것, 싫어하는 것 등)
   - 관심사와 취미
   - 과거 경험
   - 목표나 미래 계획
3. 새로운 정보를 기존 메모리와 병합하세요.
4. 메모리는 명확한 불릿 리스트 형식으로 작성하세요.
5. 새로운 정보가 기존 메모리와 충돌할 경우, 가장 최근 정보를 유지하세요.

기억하세요: 사용자가 직접적으로 언급한 사실적인 정보만 포함해야 합니다. 추측이나 추론을 하지 마세요.

아래의 채팅 기록을 바탕으로 사용자 정보를 업데이트하세요:"""

def get_memory(namespace, 
               key,
               store:BaseStore):
    """
        Des:
            현재 저장된 사용자 정보를 가져오는 함수
    """
    existing_memory = store.get(namespace=namespace,
                                key=key)
    return existing_memory.value.get('memory') if existing_memory else "현재 저장된 사용자 정보가 없습니다."
    

def node_get_response(state: MessagesState, 
                      config: RunnableConfig, 
                      store: BaseStore):
    """
        Des:
            답변 생성 노드
    """
    namespace = ("memories", config["configurable"]["user_id"])
    key = "chat_user_memory"
    existing_memory_content = get_memory(namespace=namespace, 
                                         key=key, 
                                         store=store)
    system_message = prompt_config.answer_prompt.format(memory=existing_memory_content)
    prompt = [SystemMessage(content=system_message)]+state["messages"]
    response = llm.invoke(prompt)
    return {"messages":response}

def node_write_memory(state: MessagesState, 
                      config: RunnableConfig, 
                      store: BaseStore):
    """
        Des:
            사용자 메시지를 인식하고, 개인정보로 저장하는 노드
    """
    namespace = ("memories", config["configurable"]["user_id"])
    key = "chat_user_memory"
    existing_memory_content = get_memory(namespace=namespace, 
                                         key=key, 
                                         store=store)
    system_message = prompt_config.create_memory_prompt.format(memory=existing_memory_content)
    prompt = [SystemMessage(content=system_message)]+state["messages"]
    response = llm.invoke(prompt)
    store.put(namespace=namespace, 
              key=key, 
              value={"memory":response.content})
    
builder = StateGraph(MessagesState)
builder.add_node("node_get_response", node_get_response)
builder.add_node("node_write_memory", node_write_memory)
builder.add_edge(START, "node_get_response")
builder.add_edge("node_get_response", "node_write_memory")
builder.add_edge("node_write_memory", END)
graph = builder.compile(checkpointer=ShortTermMemory,
                        store=LongTermMemory)

config = {"configurable": {"thread_id": "second_chat", 
                           "user_id": "changwoo"}}

In [9]:
graph.invoke({"messages":"안녕 나는 창우라고해"}, 
             config=config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='79d67190-8066-42e2-94a5-a0a1f393d4e2'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 79, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ecce0cd-a7fa-49d4-8471-8b53d61b4525-0', usage_metadata={'input_tokens': 79, 'output_tokens': 21, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [10]:
graph.invoke({"messages":"나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어."}, config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='79d67190-8066-42e2-94a5-a0a1f393d4e2'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 79, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ecce0cd-a7fa-49d4-8471-8b53d61b4525-0', usage_metadata={'input_tokens': 79, 'output_tokens': 21, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}

In [11]:
graph.invoke({"messages":"내 취미는 헬스장에서 운동하는거고, 돼지고기를 제일좋아해"}, config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='79d67190-8066-42e2-94a5-a0a1f393d4e2'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 79, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ecce0cd-a7fa-49d4-8471-8b53d61b4525-0', usage_metadata={'input_tokens': 79, 'output_tokens': 21, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}

In [12]:
graph.invoke({"messages":"나에 대해서 알고있는게 있어?"}, config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='79d67190-8066-42e2-94a5-a0a1f393d4e2'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 79, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ecce0cd-a7fa-49d4-8471-8b53d61b4525-0', usage_metadata={'input_tokens': 79, 'output_tokens': 21, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}

In [13]:
graph.invoke({"messages":"아니야. 잘못말했어. 내이름은 김호원이고, 나이는 54살이야. 현재 스마트엠투엠 대표로 일하고있어."}, config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='79d67190-8066-42e2-94a5-a0a1f393d4e2'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 79, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ecce0cd-a7fa-49d4-8471-8b53d61b4525-0', usage_metadata={'input_tokens': 79, 'output_tokens': 21, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}

In [14]:
graph.invoke({"messages":"나에 대해서 알고있는게 있어?"}, config)

{'messages': [HumanMessage(content='안녕 나는 창우라고해', additional_kwargs={}, response_metadata={}, id='79d67190-8066-42e2-94a5-a0a1f393d4e2'),
  AIMessage(content='안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 79, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ecce0cd-a7fa-49d4-8471-8b53d61b4525-0', usage_metadata={'input_tokens': 79, 'output_tokens': 21, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.', additional_kwargs={}, response_metadata={}